# Helm: Kubernetes, Grafana, Prometheus (Container Observability)

## Install Minikube and start Minikube cluster

In [ ]:
%%sh
## Install Minkinube
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube
# Start Minikube
minikube start

In [ ]:
!rm -rf mi*

In [ ]:
!minikube status

In [ ]:
!kubectl cluster-info

## Kubernetes Manifest

### NGINX Deployment

Reference: https://kubernetes.io/docs/concepts/workloads/controllers/deployment

In [ ]:
### nginx-deployment.yaml

In [ ]:
%%writefile nginx-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  labels: 
    app: nginx
spec:
  replicas: 3
  selector: 
    matchLabels:
      app: nginx
  template:
    metadata: 
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80

In [ ]:
# Apply deployment
!kubectl apply -f nginx-deployment.yaml

In [ ]:
!kubectl get all

In [ ]:
!kubectl expose deployment/nginx-deployment

In [ ]:
!kubectl port-forward svc/nginx-deployment 8000:80 --address 0.0.0.0

### Notepad Deployment

#### Notepad App (Streamlit and sqlite3)

In [ ]:
# Install streamlit
!pip install streamlit -q

In [ ]:
# Import libraries
import sqlite3
import streamlit as st

In [ ]:
%%writefile app.py
# Import libraries
import streamlit as st
import sqlite3
import os
import time

# Header
st.write("# Notepad :notebook: v2")
st.caption("""
            Add your thoughts here! It will be stored in a database! \n
            :warning: :red[Do not add sensitive data].
            """)
st.subheader("",divider="rainbow")

# Variable
database_name = "notes.db"

# Create a database
if database_name in os.listdir():
    con = sqlite3.connect(database_name)
    cur = con.cursor()
else:
    con = sqlite3.connect(database_name)
    # Database Cursor (cur)
    cur = con.cursor()
    cur.execute("CREATE TABLE notes(name, note, time)")
    con.commit()

# Inputs
name = st.text_input("Your Name here")
note = st.text_area("Add Note here",)
if st.button("Add a note"):
    time = time.strftime("Date: %Y-%m-%d | Time: %H:%M:%S UTC")
    st.write(f""" \n
            ### :pencil: {note} \n
            :man: {name} \n
            :watch: {time}""")
    st.success("Successful Added.")
    # st.balloons()
    ### Insert into adatabase
    cur.execute(f"""
            INSERT INTO notes VALUES
            ("{name}", "{note}", "{time}")
            """)
    con.commit()
    
# Previous Notes 
st.subheader("",divider="rainbow")
st.write("### *Previous Notes*")
# Write the data
result = cur.execute("""
                    SELECT * 
                    FROM notes
                    ORDER BY time DESC
                    """)
for name, note, time in result.fetchall():
    st.write(f""" \n
            #### :pencil: {note} \n
            :man: Written By: **{name}** \n
            :watch: {time}""")
    st.subheader("",divider="gray")
    
# Close Connection
con.close()

In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit -q
COPY app.py .
EXPOSE 8501
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0"]

In [ ]:
# Docker login
# docker login

In [ ]:
DOCKER_USERNAME=""

In [ ]:
# Build an image
!docker build -t $DOCKER_USERNAME/app . -q

In [ ]:
# List images
!docker images

In [ ]:
!docker push $DOCKER_USERNAME/app

In [ ]:
# Remove all docker ps
# !docker rm -f $(docker ps -aq)

In [ ]:
# Create a firewall (GCP)
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:8501,tcp:8000,tcp:9000 --source-ranges=0.0.0.0/0

#### notepadd-deployment.yaml

In [ ]:
### notepadd-deployment.yaml

In [ ]:
%%writefile notepad-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: notepad-deployment
  labels: 
    app: notepad
spec:
  replicas: 3
  selector: 
    matchLabels:
      app: notepad
  template:
    metadata: 
      labels:
        app: notepad
    spec:
      containers:
      - name: notepad
        image: mattregojos/app:latest
        ports:
        - containerPort: 8501

In [ ]:
# Create notepad namespace
!kubectl create namespace notepad

In [ ]:
# Apply deployment
!kubectl apply -f notepad-deployment.yaml -n notepad

In [ ]:
!kubectl get all -n notepad

In [ ]:
!kubectl expose deployment.apps/notepad-deployment -n notepad

In [ ]:
!kubectl get all -n notepad

In [ ]:
!kubectl port-forward service/notepad-deployment 8501:8501 --address 0.0.0.0 -n notepad

In [ ]:
!kubectl get pods -n notepad

## Helm: Kubernetes, Grafana, Prometheus

### Install Helm

In [ ]:
!wget https://get.helm.sh/helm-v3.13.1-linux-amd64.tar.gz

In [ ]:
!tar -zxvf helm-v3.13.1-linux-amd64.tar.gz

In [ ]:
!sudo mv linux-amd64/helm /usr/local/bin/helm

In [ ]:
%%sh 
# Remove linux-amd64 and helm-v*
rm -rf linux-amd64
rm helm-v*

In [ ]:
# Verify
!helm help

### Helm Artifact Hub

Resource: 
- https://helm.sh
- https://artifacthub.io

#### Kube-Promethus-Stack

In [ ]:
!helm repo add prometheus-community https://prometheus-community.github.io/helm-charts

In [ ]:
# I add namespace monitoring
!kubectl create namespace monitoring

In [ ]:
!helm install my-kube-prometheus-stack prometheus-community/kube-prometheus-stack --version 52.1.0 -n monitoring

In [ ]:
!helm list -n monitoring

In [ ]:
!kubectl get all -n monitoring

In [ ]:
# Create a firewall
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:8000,tcp:9000,tcp:10000 --source-ranges=0.0.0.0/0

In [ ]:
# Port-forward
!kubectl port-forward svc/my-kube-prometheus-stack-grafana  8000:80 -n monitoring --address 0.0.0.0 

In [ ]:
# User: Admin
# Password: prom-operator